# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
# Imports
import configparser
import math

from matplotlib import pyplot as plt
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20)})


In [ ]:
def load_result_list(experiments):
    strategies = []
    networks = []
    
    score = {}
    correct = {}
    distance = {}
    initial_distance = {}
    
    for experiment_id in experiments:
        experiment, exp_score, exp_correct, exp_distance, exp_initial = load_results(experiment_id)
        
        exp_strategies = experiment.config.get('strategies', 'enabled').split(',')
        for i, s in enumerate(exp_strategies):
            
            if s not in strategies:
                strategies.append(s)
                score[s] = {}
                correct[s] = {}
                distance[s] = {}
                initial_distance[s] = {}
            
            exp_networks = experiment.config.get('networks', 'enabled').split(',')
            for j, n in enumerate(exp_networks):
                
                if n not in networks:
                    networks.append(n)
                
                score[s][n] = exp_score[i][j]
                correct[s][n] = exp_correct[i][j]
                distance[s][n] = exp_distance[i][j]
                if exp_initial is not None:
                    initial_distance[s][n] = exp_initial[i][j]
    
    return experiment, strategies, networks, score, correct, distance, initial_distance
    
def load_results (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    score = np.load(experiment.get_filename('score.npy'))
    correct = np.load(experiment.get_filename('correct.npy'))
    distance_score = np.load(experiment.get_filename('distance_score.npy'))
    # Older code did not produce the below data
    try:
        initial_distance = np.load(experiment.get_filename('initial_distance.npy'))
    except FileNotFoundError:
        initial_distance = None
    return experiment, score, correct, distance_score, initial_distance

def individual_title(config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'Hill-Climb'
        else:
            title_individual = 'Mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' Serial-Indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' Fallback-Indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' Parallel-Indiv.'
    else:
        title_individual = 'social-only'
    return title_individual

def result_title (config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'hill-climb'
        else:
            title_individual = 'mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' serial indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' fallback indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' parallel indiv.'
    else:
        title_individual = 'social-only'
    title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
        config.getint('abm', 'N'),
        config.getint('abm', 'bit_count'),
        config.getint('abm', 'K'),
        config.getfloat('abm', 'nk_exponent', fallback=1),
        config.getint('strategies', 'sample', fallback=None),
        {True: 'critical', False: 'non-critical'}[config.getboolean('strategies', 'critical')],
        title_individual)
    return title


In [ ]:
symbols = ['.','.','.','.']
colors = ['#3f3fff','#3f3fff','#3f3fff','#3f3fff']
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']
reproduce = [
    '2020-12-09 140742 a5a2be3'
]

experiment, learning_strategies, networks, score, correct, distance_score, initial_distance = load_result_list(reproduce)
config = experiment.config

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

for i, strat in enumerate(learning_strategies):
    plt.subplot(len(learning_strategies), 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        xlabels.append(net)
        y = score[strat][net][0]
        best = max(y)
        ys.append(best)
        converge_threshold = 0.99 * best
        converge_time = None
        for t, y_t in enumerate(y):
            if y_t >= converge_threshold:
                converge_time = t
                break
        err95 = 1.96 * np.array(score[strat][net][1]) / math.sqrt(runs)
        err95s.append(err95[t])
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=16)
    plt.errorbar(ys, range(len(xlabels)), xerr=err95s, fmt='{}'.format(symbols[i]), markerfacecolor='{}ff'.format(colors[i]), color=colors[i], markeredgecolor='{}ff'.format(colors[i]), label=strat, markersize=12)
    plt.xlim([0, 1])
    plt.grid(True)
    plt.yticks(ticks=range(len(xlabels)), labels=xlabels, rotation=0, fontsize=16)
    plt.xticks(fontsize=16)
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

#plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), fontsize=12)
plt.tight_layout()
out = experiment.get_filename('performance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('performance.eps'), dpi=600)


In [ ]:
plt.figure(figsize=(30, 18))
for i, title in enumerate(learning_strategies):
    for j, title2 in enumerate(networks):
        plt.subplot(num_strat, num_net, 1 + i * num_net + j)

        y = score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'k-')     

        y = correct[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(correct[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'b:')
        
        y = distance_score[i][j][0]
        x = range(len(y))
        err95 = 1.96 * np.array(distance_score[i][j][1]) / math.sqrt(runs)
        plt.fill_between(x, y - err95, y + err95, color="#aaaaff7f")
        plt.plot(x, y, 'g--')
        
        plt.title(f'{title} - {title2}')
        if j == 0:
            plt.ylabel('Frac. Correct')
        if i == 2:
            plt.xlabel('Time Step')
        plt.grid(True)
        plt.tight_layout()
        plt.ylim([0.0, 1])

out = experiment.get_filename('plot.png')
plt.savefig(out, dpi=300)

In [ ]:
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']

for i, strat in enumerate(learning_strategies):
    plt.subplot(len(learning_strategies), 1,1+i)
    xlabels = []
    ys = []
    err95s = []
    for j, net in enumerate(reversed(networks)):
        if net in exclude:
            continue
        y = score[strat][net][0]
        err95 = 1.96 * np.array(initial_distance[strat][net][1]) / math.sqrt(runs)
        plt.plot(range(len(y)), y, label=net)
    plot_title = strat 
    if plot_title == 'Local Majority':
        plot_title = 'Bitwise Majority'
    plot_title += ' + ' + individual_title(experiment.config)
    plt.title(plot_title, fontsize=16)
    plt.grid(True)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.legend(loc='lower right', fontsize=12)
#    if i == 0:
#plt.ylabel('Mean Final Score', fontsize=20)

plt.tight_layout()
out = experiment.get_filename('initial_distance.png')
plt.savefig(out, dpi=600)
plt.savefig(experiment.get_filename('initial_distance.eps'), dpi=600)
